In [72]:
#The benchmark accuracy to beat is 65.5% which is the accuracy one gets if the 
#higher ranked player is always predicted to win.

In [4]:
#https://www.kaggle.com/code/afafathar3007/atp-matches-eda-stats-prediction

import csv
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [5]:
df = pd.read_csv(r'C:\Users\abeck\OneDrive\Desktop\atp_matches_2022.csv')

In [24]:
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2022-8888,Atp Cup,Hard,16,A,20220103,300,200000,NaN,NaN,...,50.0,32.0,7.0,10.0,3.0,5.0,11.0,3308.0,19.0,2260.0
1,2022-8888,Atp Cup,Hard,16,A,20220103,299,133430,NaN,NaN,...,33.0,21.0,8.0,9.0,3.0,6.0,14.0,2475.0,20.0,2230.0
2,2022-8888,Atp Cup,Hard,16,A,20220103,298,105138,NaN,NaN,...,80.0,62.0,20.0,16.0,6.0,7.0,19.0,2260.0,9.0,3706.0
3,2022-8888,Atp Cup,Hard,16,A,20220103,297,105807,NaN,NaN,...,27.0,17.0,1.0,7.0,4.0,8.0,20.0,2230.0,860.0,18.0
4,2022-8888,Atp Cup,Hard,16,A,20220103,296,106421,NaN,NaN,...,35.0,22.0,4.0,8.0,3.0,7.0,2.0,8640.0,11.0,3308.0


In [9]:
len(df)

2917

In [20]:
# Sanity checking tourney_id
ct = 0

for index, row in df.iterrows():
    if (row['tourney_id'] == ""):
        ct = ct + 1
print(ct)

2917


In [18]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [21]:
# Sanity checking tourney_name
ct = 0

for index, row in df.iterrows():
    if (row['tourney_name'] == ""):
        ct = ct + 1
print(ct)

0


In [22]:
# Sanity checking surface
ct = 0

for index, row in df.iterrows():
    if (row['surface'] == ""):
        ct = ct + 1
print(ct)

0


In [23]:
# Sanity checking draw_size
ct = 0

for index, row in df.iterrows():
    if (row['draw_size'] == ""):
        ct = ct + 1
print(ct)

0


In [25]:
df['winner_name']

0       Felix Auger Aliassime
1            Denis Shapovalov
2       Roberto Bautista Agut
3         Pablo Carreno Busta
4             Daniil Medvedev
                ...          
2912       Aristotelis Thanos
2913         Petros Tsitsipas
2914       Hong Kit Jack Wong
2915             Coleman Wong
2916         Sylvestre Monnou
Name: winner_name, Length: 2917, dtype: object

In [27]:
#As first step, let's clean the data and retain only important features for our prediction task.
#We proceed to the removal of features that are redundant/ crrelated with each other, 
#and those that are obviously not available before a match to predict.
#"winner_name", "loser_name" & "tourney_name" because they are almost redundant and replaceable by respectively "winner_id", "loser_id" & "tourney_id".
#"minutes", "score", "1stIn", "_1stWon" ,"SvGms", "l_bpFaced", "l_bpSaved", "svpt" features because cannot be known in advance.

df = df.drop(columns=['score','tourney_name','winner_name',
                      'loser_name','minutes', 'l_1stIn', 'l_1stWon', 'l_2ndWon','l_ace', 'l_svpt',
                      'l_SvGms','l_bpFaced','l_df', 'l_bpSaved',  'w_1stIn', 'w_1stWon','w_2ndWon', 'w_SvGms',
                      'w_ace', 'w_svpt','w_bpFaced', 'w_bpSaved','w_df','winner_rank_points', 'loser_rank_points'])
print("remained features ", df.columns)

remained features  Index(['tourney_id', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'match_num', 'winner_id', 'winner_seed', 'winner_entry', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_hand', 'loser_ht', 'loser_ioc', 'loser_age',
       'best_of', 'round', 'winner_rank', 'loser_rank'],
      dtype='object')


In [28]:
none_percent = df.isnull().sum() * 100 / len(df)
none_values_df = pd.DataFrame({'Feature name': df.columns, 'None values count': df.isnull().sum(),
                                 'None values percent': none_percent})
print(none_values_df.reset_index().drop(columns=['index']))

     Feature name  None values count  None values percent
0      tourney_id                  0             0.000000
1         surface                  0             0.000000
2       draw_size                  0             0.000000
3   tourney_level                  0             0.000000
4    tourney_date                  0             0.000000
5       match_num                  0             0.000000
6       winner_id                  0             0.000000
7     winner_seed               1697            58.176208
8    winner_entry               2489            85.327391
9     winner_hand                  0             0.000000
10      winner_ht                 69             2.365444
11     winner_ioc                  0             0.000000
12     winner_age                  2             0.068564
13       loser_id                  0             0.000000
14     loser_seed               2212            75.831334
15    loser_entry               2263            77.579705
16     loser_h

In [29]:
#Remove rows with > 85 missing

features_to_remove = none_values_df[none_values_df["None values percent"] > 85]
print(features_to_remove)
df = df.drop(columns=features_to_remove['Feature name'].values)

              Feature name  None values count  None values percent
winner_entry  winner_entry               2489            85.327391


In [30]:
#Label numeric columns

numeric_columns = ['winner_rank', 'loser_rank', 'winner_age', 'loser_age', 'winner_ht', 'loser_ht']
df[numeric_columns] = df[numeric_columns].astype(float)

In [31]:
#"tourney_date" is in the format of YYYYMMDD
df['tourney_year'] = df.tourney_date.astype(str).str[:4].astype(int)
df['tourney_month'] = df.tourney_date.astype(str).str[4:6].astype(int)

#Now drop "tourney_date"
df = df.drop(columns=['tourney_date'])

### Feature Engineering

In [32]:
df = df.rename(columns={"loser_age": "first_age", 
                        "loser_entry": "first_entry", 
                        "loser_hand": "first_hand",
                        "loser_ht": "first_ht", 
                        "loser_id": "first_id", 
                        "loser_ioc": "first_ioc",
                        "loser_rank": "first_rank", 
                        "loser_rank_points": "first_rank_points",
                        "loser_seed": "first_seed",
                        "winner_age": "second_age", 
                        "winner_entry": "second_entry",
                        "winner_hand": "second_hand",
                        "winner_ht": "second_ht", 
                        "winner_id": "second_id", 
                        "winner_ioc": "second_ioc",
                        "winner_rank": "second_rank", 
                        "winner_rank_points": "second_rank_points",
                        "winner_seed": "second_seed",
                       },)

Feature engineering For our supervised prediction model, we have to define our target feature!

We will transform our data so that we have 2 players (first player & second player), their respective personal informations (id, hand, age, etc) and general informations about the match and the tourney.

Then we will create a column "label" which is equal to 1 if player 1 wins, 0 if player 2 wins.

to do so, we will create a first copy of our dataset where the winner is considered as first player so label=0. Then a second copy where we inverse the places of the players so label=1.

First copy of the data where the loser is considered as first player and the winner as second player

### Create second copy of data with inversed positions

In [33]:
copy_2_df = df.copy()

In [34]:
copy_2_df[[ 'first_age','first_hand','first_ht','first_id','first_ioc','first_rank','first_seed',
            'second_age','second_hand','second_ht','second_id','second_ioc','second_rank','second_seed']]\
=copy_2_df[['second_age','second_hand','second_ht','second_id','second_ioc','second_rank','second_seed',
             'first_age','first_hand','first_ht','first_id','first_ioc','first_rank','first_seed']]

In [35]:
## Construct label feature
winner_player2 = np.zeros(df.shape[0]) # second player wins so label=0
df['label'] = winner_player2

winner_player1 = np.ones(copy_2_df.shape[0]) # first player wins so label=1
copy_2_df['label'] = winner_player1 

df = pd.concat([df,copy_2_df])
#shuffle data
# df = df.sample(frac=1) .reset_index(drop=True)
# df

In [71]:
#Encode categorical variables for prediction

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

hand_encoder = LabelEncoder()
df['first_hand'] =(df['first_hand'].astype(str))
df['second_hand'] = (df['second_hand'].astype(str))

df['first_ioc'] = df['first_ioc'].astype(str)
df['second_ioc'] = (df['second_ioc'].astype(str))
df['first_entry'] = (df['first_entry'].astype(str))
df['surface'] = (df['surface'].astype(str))
df['tourney_level'] = (df['tourney_level'].astype(str))
df['tourney_id'] = (df['tourney_id'].astype(str))
df['round'] = (df['round'].astype(str))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5834 entries, 0 to 2916
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tourney_id     5834 non-null   object 
 1   surface        5834 non-null   object 
 2   draw_size      5834 non-null   float64
 3   tourney_level  5834 non-null   object 
 4   match_num      5834 non-null   float64
 5   second_id      5834 non-null   float64
 6   second_seed    5834 non-null   float64
 7   second_hand    5834 non-null   object 
 8   second_ht      5834 non-null   float64
 9   second_ioc     5834 non-null   object 
 10  second_age     5834 non-null   float64
 11  first_id       5834 non-null   float64
 12  first_seed     5834 non-null   float64
 13  first_entry    5834 non-null   object 
 14  first_hand     5834 non-null   object 
 15  first_ht       5834 non-null   float64
 16  first_ioc      5834 non-null   object 
 17  first_age      5834 non-null   float64
 18  best_of 

In [47]:
from sklearn.preprocessing import LabelBinarizer
hand_encoder = LabelEncoder()
df['first_hand'] = hand_encoder.fit_transform(df['first_hand'])
df['second_hand'] = hand_encoder.transform(df['second_hand'])

df['first_ioc'] = LabelEncoder().fit_transform(df['first_ioc'])
df['second_ioc'] = LabelEncoder().fit_transform(df['second_ioc'])
df['first_entry'] = LabelEncoder().fit_transform(df['first_entry'])
df['surface'] = LabelBinarizer().fit_transform(df['surface'] )
df['tourney_level'] = LabelEncoder().fit_transform(df['tourney_level'] )
df['tourney_id'] = LabelEncoder().fit_transform(df['tourney_id'])
df['round'] = LabelEncoder().fit_transform(df['round'])

In [48]:
df.info(60)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5834 entries, 0 to 2916
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tourney_id     5834 non-null   int32  
 1   surface        5834 non-null   int32  
 2   draw_size      5834 non-null   int64  
 3   tourney_level  5834 non-null   int32  
 4   match_num      5834 non-null   int64  
 5   second_id      5834 non-null   int64  
 6   second_seed    1925 non-null   float64
 7   second_hand    5834 non-null   int32  
 8   second_ht      5617 non-null   float64
 9   second_ioc     5834 non-null   int32  
 10  second_age     5830 non-null   float64
 11  first_id       5834 non-null   int64  
 12  first_seed     1925 non-null   float64
 13  first_entry    5834 non-null   int32  
 14  first_hand     5834 non-null   int32  
 15  first_ht       5617 non-null   float64
 16  first_ioc      5834 non-null   int32  
 17  first_age      5830 non-null   float64
 18  best_of 

In [49]:
df.head(20)

,tourney_id,surface,draw_size,tourney_level,match_num,second_id,second_seed,second_hand,second_ht,second_ioc,...,first_ht,first_ioc,first_age,best_of,round,second_rank,first_rank,tourney_year,tourney_month,label
0,102,0,16,0,300,200000,NaN,1,193.0,3,...,183.0,16,33.7,3,0,11.0,19.0,2022,1,0.0
1,102,0,16,0,299,133430,NaN,0,185.0,3,...,188.0,16,30.4,3,0,14.0,20.0,2022,1,0.0
2,102,0,16,0,298,105138,NaN,1,183.0,16,...,196.0,49,24.8,3,7,19.0,9.0,2022,1,0.0
3,102,0,16,0,297,105807,NaN,1,188.0,16,...,NaN,49,25.1,3,7,20.0,860.0,2022,1,0.0
4,102,0,16,0,296,106421,NaN,1,198.0,53,...,193.0,3,21.4,3,7,2.0,11.0,2022,1,0.0
5,102,0,16,0,295,133430,NaN,0,185.0,3,...,185.0,53,24.4,3,7,14.0,167.0,2022,1,0.0
6,102,0,16,0,294,134770,NaN,1,183.0,44,...,180.0,55,31.5,3,6,8.0,33.0,2022,1,0.0
7,102,0,16,0,293,105936,NaN,1,185.0,55,...,185.0,44,24.7,3,6,42.0,345.0,2022,1,0.0
8,102,0,16,0,292,106426,NaN,1,185.0,4,...,180.0,55,31.5,3,6,17.0,33.0,2022,1,0.0
9,102,0,16,0,291,105936,NaN,1,185.0,55,...,188.0,4,24.5,3,6,42.0,139.0,2022,1,0.0


In [51]:
## let's handle the few remaing None values using SimpleImputer
from sklearn.impute import SimpleImputer
df_imputed = pd.DataFrame(SimpleImputer().fit_transform(df))
df_imputed.columns = df.columns
df_imputed.index = df.index
df = df_imputed.copy()

In [52]:
print('Final shape of data after cleaning and preprocessing: ', df.shape)

Final shape of data after cleaning and preprocessing:  (5834, 25)


Let's now train and execute our prediction model (XGBoost)

In [53]:
y = df['label']
df_X = df.drop(columns='label')

In [62]:
from sklearn.model_selection import train_test_split

# split data : 80% for train and 20% for test.
X_train, X_test, y_train, y_test = train_test_split(df_X, y, test_size=0.2)

In [63]:
#pip install scikit-learn

XGBoost classifier Like Random Forests, Extreme Gradient Boosting is a set of decision trees. The two main differences are:

How trees are built: random forests builds each tree independently while gradient boosting builds one tree at a time. This additive model (ensemble) works in a forward stage-wise manner, introducing a weak learner to improve the shortcomings of existing weak learners. Combining results: random forests combine results at the end of the process (by averaging or "majority rules") while gradient boosting combines results along the way.

In [65]:
import xgboost as xgb
from xgboost import XGBClassifier

#Call the classifier
XGB_classifier = XGBClassifier()
#fit the data
XGB_classifier.fit(X_train, y_train)
#predict 
XGB_predictions = XGB_classifier.predict(X_test)

In [70]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


print('Confusion matrix')
print(confusion_matrix(y_test,XGB_predictions))
print('Classification report')
print(classification_report(y_test,XGB_predictions))
print('Accuracy= ', accuracy_score(y_test, XGB_predictions))

Confusion matrix
[[400 185]
 [201 381]]
Classification report
              precision    recall  f1-score   support

         0.0       0.67      0.68      0.67       585
         1.0       0.67      0.65      0.66       582

    accuracy                           0.67      1167
   macro avg       0.67      0.67      0.67      1167
weighted avg       0.67      0.67      0.67      1167

Accuracy=  0.6692373607540703
